# Train mobilenet image classifier
Currenly uses mobilenet V2 to train an image classifier.

Future work:  
[] support different deep learning neural network architectures (like VGG, ResNet, ...)  
[] expose hyper parameters of those architectures as input to this component  
[] support hyperparameter tuning (ideally using Katib)  

In [ ]:
import os
import sys
#os.environ['create_image']='True'
os.environ['repository']='romeokienzler'
os.environ['version']='0.15'
#
#os.environ['install_requirements']='True'

In [2]:
if bool(os.environ.get('create_image',False)):

    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat tensorflow==2.9.1
    ADD train-mobilenet_v2.ipynb /
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)
        

    !docker build -t claimed-train-mobilenet_v2:`echo $version` .
    !docker tag claimed-train-mobilenet_v2:`echo $version` `echo $repository`/claimed-train-mobilenet_v2:`echo $version`
    !docker push `echo $repository`/claimed-train-mobilenet_v2:`echo $version`
    !rm Dockerfile
    sys.exit(0)
elif bool(os.environ.get('install_requirements',False)):
    !pip install tensorflow==2.9.1

Sending build context to Docker daemon  96.26kB
Step 1/3 : FROM registry.access.redhat.com/ubi8/python-39
 ---> a531ae76755e
Step 2/3 : RUN pip install ipython nbformat tensorflow==2.9.1
 ---> Using cache
 ---> ebc26374c664
Step 3/3 : ADD train-mobilenet_v2.ipynb /
 ---> 0a65716cf0cf
Successfully built 0a65716cf0cf
Successfully tagged claimed-train-mobilenet_v2:0.15
The push refers to repository [docker.io/romeokienzler/claimed-train-mobilenet_v2]

5f91aec9: Preparing 
7d7d69b5: Preparing 
2e0f4ef5: Preparing 
276847a2: Preparing 
534f4e1b: Preparing 
c926eef9: Preparing 
5f91aec9: Pushed lready exists 3kBA0.15: digest: sha256:a7233cfea658b1c5babd6d65501de723e15d4d10f9f721a1929bf65007988e21 size: 1795


SystemExit: 0

/home/romeokienzler/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import os
import tensorflow as tf
# from tensorflow.keras.applications import ResNet50V2 # , MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
from tensorflow import keras
import os.path
import glob
import sys
import logging

In [ ]:
# @dependency codait_utils.ipynb
# @param image_shape shape the images shall be scaled to and the
# model then will accept
# @param model zip file name
# @param data zip file name
# @param model folder name
# @param data folder name
# @param epochs number of epochs to train
# @param checkpoint activate checkpointing
# @param checkpoint_ip minio endpoint
# @param checkpoint_user minio user
# @param checkpoint_pass minio pw
# @param checkpoint_bucket minio bucket
# @returns model zip file

In [ ]:
image_shape = os.environ.get('image_shape', '400,400')
image_folder = os.environ.get('image_folder', 'images')
model_folder = os.environ.get('model_folder', 'model')
batch_size = int(os.environ.get('batch_size', 32))
optimizer = os.environ.get('optimizer', 'adam')
loss = os.environ.get('loss', 'categorical_crossentropy')
data_dir = os.environ.get('data_dir', '.')
epochs = int(os.environ.get('epochs', 1))

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))


for parameter in parameters:
    exec(parameter)
    logging.warning('Parameter: ' + parameter)

batch_size = int(batch_size)


for parameter in parameters:
    exec("logging.warning('final parameter: ' + str({}))".format(parameter.split('=')[0]))
    exec("logging.warning('final parameter type: ' + str(type({})))".format(parameter.split('=')[0]))

In [ ]:
#data_dir = '../../data/'
#batch_size = 4

In [ ]:
folder = glob.glob(data_dir + image_folder + "/*")
num_classes = len(folder)

In [ ]:
input_shape = 'dummy'  # make the compiler happy
exec('input_shape = (' + image_shape + ')')

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir + image_folder,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=input_shape,
    batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir + image_folder,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=input_shape,
    batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))

In [ ]:
def my_net(model, num_classes, freeze_layers=10, full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes, activation='softmax')(x)
    model_final = Model(model.input, out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [ ]:
# model = ResNet50V2(weights='imagenet',include_top=False)
# model = my_net(model)

In [ ]:
exec('input_shape = (' + image_shape + ',3)')

model = tf.keras.applications.MobileNetV2(
    input_shape=input_shape, alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=num_classes,
    classifier_activation='softmax'
)
model = my_net(model, num_classes=num_classes)

In [ ]:
# model = tf.keras.applications.VGG16(
#     include_top=True, weights=None, input_tensor=None,
#     input_shape=(244, 244, 3), pooling=None, classes=2,
#     classifier_activation='softmax'
# )
# model = my_net(model)

In [ ]:
logging.warning('compiling model')


model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [ ]:
class MetricLogger(keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        logging.warning('Train-accuracy={}'.format(logs['accuracy']))
        logging.warning('Validation-accuracy={}'.format(logs['val_accuracy']))


In [ ]:
logging.warning('training model')


model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_ds,
    callbacks=[MetricLogger()]
)

In [ ]:
logging.warning('saving model')

model.save(data_dir + model_folder)

logging.warning('done, exiting')
